# Record based Finite Automaton for TTR

# Code execution and explanation:

In [2]:
'''
Initializing all pre-requisite packages.
'''
import os, copy
import  tkinter as tk
from automata.fa.nfa import NFA
from visual_automata.fa.nfa import VisualNFA
from tkinter import font as tkFont
from tkinter import ttk
from tkinter import font as tkFont
from itertools import chain, combinations
from PIL import ImageTk, Image
from pdf2image import convert_from_path


'''
SECTION 1:
1. Visualization & Superposition of records based on events in the form of automaton.
2. Projection via Allen Composition.
'''

'''
PART 1A:
Function definitions to define a NonDterministic Finite Automaton to store records of events.
'''
class NFA:
    #Constructor class
    def __init__(self):
        self.init = "q0" #initalState(str)
        
        self.final = set()  #finalState(set)
        
        self.trans = {}  # dict with key = previousState (number) containing a dict as value
                         #      with key = act 
                         #           val = set of poststates (numbers)
    
    
    #To accept the action based transition or not.
    def accept(self,strList):
        if len(strList) == 0:
            if self.init in self.final:
                ans = "accepted"
            else:
                ans = "rejected"
        else:
            stateList = [self.init]
            ans = "rejected"
            for state in self.steps(strList, stateList):
                if state in self.final:
                    ans = "accepted"
                    break
        return ans
    
    # To iterate over each state in the input list.
    def steps(self, strList, stateList):
        if len(strList) == 1:
            temp = self.step(strList[0],stateList)
        elif len(strList) > 1:
            symbol = strList[0]
            tail = strList[1:]
            temp = self.steps(tail, self.step(symbol,stateList))
        return(temp)
    
    # To check if the path exist to terminal state by appending all child state of the current state considered.  
    def step(self, symbol, stateList):
        temp = []
        for state in stateList:
            if state in self.trans:
                if symbol in self.trans[state]:
                    for next in self.trans[state][symbol]:
                        temp.append(next)
        return(temp)
    
    
    def add_trans(self, pre, label, post):
        if pre in self.trans:
            if label in self.trans[pre]:
                self.trans[pre][label].add(post)
            else:
                self.trans[pre][label] = {post}
        else:
            self.trans[pre] = {label: {post}}

    def add_final(self, state):
        self.final.add(state)

    def trace(self,strList):  # depth-first, contra breadth-first accept above
        return self.dfsRun([(strList,self.init,[])])
                
    def dfsRun(self,frontier):
        if frontier == []:
            return "rejected" 
        else:
            strList = frontier[-1][0]
            state = frontier[-1][1]
            run = frontier.copy()[-1][2]
            frontier.pop()
            if len(strList) == 0 and state in self.final:
                return run
            elif len(strList)>0:
                symbol = strList[0]
                tail = strList[1:]
                for next in self.step(symbol,[state]):
                    r = run.copy()
                    r.append((symbol,next))
                    frontier.append((tail,next,r))
            return self.dfsRun(frontier)
    
    #To accept the State based transition or not.
    def acceptState(self,strList):
        temp = []
        tempStore = []
        ans = "rejected"
        for i in strList:
            for key, value in self.stateD.items():
                if i == str(value):
                    temp.append(key)
        for i in range(len(temp)):
            start = temp[0]
            upcome = temp[1:]
            if start in self.trans:
                for value in self.trans[start].values():
                    try:
                        if (value.pop()) == upcome[0]:
                            temp = upcome
                    except:
                        break
        if self.final == set(temp):
            ans = "accepted"
        return ans
    
    # Converting the finite automaton into its S-word counterpart.
    def make_sfa(self,label):
        temp = sfa()
        temp.labels = { label }
        invStateD = { self.init : 0 }
        istate = 1
        for i in self.trans: #iterate over prevState
            if not(i in invStateD):
                invStateD[i] = { istate }
                istate += istate 
            temp.trans[invStateD[i]] = {}
            poststates = set()
            for act in self.trans[i]: #iterate over actions
                temp.actD[s2s({act})] ={ act }
                for post in self.trans[i][act]: #iterate over postState 
                    if not(post in invStateD):
                        invStateD[post] = istate
                        istate += istate        
                    poststates.add(invStateD[post])
                temp.trans[invStateD[i]] = dicMerge(temp.trans[invStateD[i]],
                                                    { s2s({act}): poststates })
        for i in invStateD:
            temp.stateD[invStateD[i]] = { label : i }
            if i in self.final:
                temp.final.add(invStateD[i])
        return temp

'''
Initializing Child class sfa of parent class NFA.
'''    
    # sfa = set/structured finite automata (for S-words)
    # nfa with trans given for states and acts described by dictionaries 
    # stateD decoding/structuring state as a dictionary [record]
    # actD decoding str(act) [better: s2s(act)] to expose the set behind act
    # plus
    # labels used by stateD to describe states
    # make disjoint from other sfa to keep information private
    # and method 'acc' for accept given a list of sets (internally representing
    #                                   a set qua symbol as the string from s2s)

class sfa(NFA):

#  init = 0
#  stateD = {}
#  actD = {}
#  labels = { "need to fix for use in stateD" }                 
    
    # Initating the Constructor function.
    def __init__(self):
        super().__init__()
        self.init = 0
        self.final = set()
        self.stateD = { 0:{} }
        self.actD = {}
        self.labels = { "need to fix for use in stateD" }                 
    
    # To check for the acceptance of actions performed in the automaton.    
    def acc(self,listSet):
        return self.accept(sfaInput(listSet))
    
    # To check for the acceptance of sequence of states in the automaton.
    def mapper(self,st):
        return self.acceptState(stateInput(st))  
    
    def voc(self):
        temp = set()
        for a in self.actD:
            temp.update(self.actD[a])
        return temp

# To convert input listSet of state representation to string format.
def stateInput(listSet):
    temp = []
    for i in listSet:
        temp.append(str(i))
    return temp

# To convert input listset of action representation to string format.
def sfaInput(listSet):
    temp = []
    for i in listSet:
        temp.append(s2s(i))
    return temp
    
# The variant of str(set0) s.t. s2s({-1,-3,-2}) == s2s({-2,-3,-1})
def s2s(set0):
    temp = []
    for i in set0:
        temp.append(i)
    temp.sort()
    return str(temp)

# To merge two dict s-word automaton representation.
def dicMerge(dic1,dic2):
    temp = dic1.copy()
    for key in dic2:
        if key in dic1:
            temp[key] = temp[key].union(dic2[key])
        else:
            temp[key] = dic2[key]
    return temp
    
# To encode interval a in action (public) but not in state (private)
def uld(a):   
    temp = NFA()
    temp.init = "u"      # + "(" + str(a) + ")"
    temp.final = {"d"}   # + "(" +str(a) + ")"}
    la = "l"             # +"("+str(a)+")"
    temp.trans = { temp.init: {a: {la}},
                   la: {-a: temp.final}  }
    return temp

'''
Function Definition to superpose two definite automaton of s-words.
'''

# Main function definition to superpose two automatons.
def sup(sfa1,sfa2):     
    voc1 = sfa1.voc()
    voc2 = sfa2.voc()
    temp = sfa()
    temp.stateD = { 0: {1:sfa1.init, 2:sfa2.init} }
    newp = {(sfa1.init,sfa2.init)}
    done = set()
    while not(newp==set()):
        nextp = set()
        for q in newp:
            morecu = rulecu(sfa1,sfa2,voc1,voc2,q,temp)
            mored1 = ruled1(sfa1,voc2,q,temp)
            mored2 = ruled2(sfa2,voc1,q,temp)        
            morep = morecu.union(mored1.union(mored2))
            nextp.update(morep)
        done.update(newp)
        newp = nextp.difference(done)
    for i in temp.stateD:
        if temp.stateD[i][1] in sfa1.final and temp.stateD[i][2] in sfa2.final:
            temp.final.add(i)
    temp.stateD = unwindSD(temp.stateD,sfa1.stateD,sfa2.stateD)
    temp.labels = sfa1.labels.union(sfa2.labels)
    return temp

#  To unwind temp.stateD using sfa1, sfa2
def unwindSD(sD,sD1,sD2):
    temp = {}
    for i in sD:
    # flatten:        
        temp[i] = dicUnion(sD1[sD[i][1]],sD2[sD[i][2]])
    return temp

# To perform union of two dictionary.
def dicUnion(dic1,dic2):
    temp = dic1.copy()
    for key in dic2:
        if key in temp:
            if not(dic2[key] == temp[key]):
                print("Clashing keys/labels (dicUnion)")
                print("key,dic2[key],temp[key] = ", key,dic2[key],temp[key])
                temp[key] = dicUnion0(temp[key],dic2[key])
        else:
            temp[key] = dic2[key]
    return temp

def dicUnion0(dic1,dic2):
    temp = dic1.copy()
    for key in dic2:
        if key in temp:
            if not(dic2[key] == temp[key]):
#                print("Clashing keys/labels (dicUnion0)")
#                print("key,dic2[key],temp[key] = ", key,dic2[key],temp[key])
                temp[key].update(dic2[key])
        else:
            temp[key] = dic2[key]
    return temp

# To convert string to list
def listify(arg):
    return arg if isinstance(arg, list) else [arg]

# To add cu to p [side-effects on temp]        
def rulecu(sfa1,sfa2,voc1,voc2,q,temp): 
    morep = set()
    moretrans = {}
    if (q[0] in sfa1.trans  and  q[1] in sfa2.trans):
        for a1 in sfa1.trans[q[0]]:
            for a2 in sfa2.trans[q[1]]:
                s1 = sfa1.actD[a1]
                s2 = sfa2.actD[a2]
                s = s1.union(s2)
                if contain(s1,voc2,s2) and contain(s2,voc1,s1):
                    for r1 in sfa1.trans[q[0]][a1]:
                        for r2 in sfa2.trans[q[1]][a2]:
                            r = (r1,r2)
                            morep.add(r)
                            temp.actD[s2s(s)] = s
                            add2stateD(temp,r)
                            qcode = decodeState(temp,q)
                            di = {s2s(s): {decodeState(temp,r)}} 
                            if qcode in temp.trans:
                                temp.trans[qcode] = dicMerge(temp.trans[qcode],
                                                             di)
                            else:
                                temp.trans[qcode] = di
    return morep

# To add d1 to p [side-effects on temp]
def ruled1(sfa1,voc2,q,temp): 
    morep = set()
    moretrans = {}
    if q[0] in sfa1.trans:
        for a1 in sfa1.trans[q[0]]:
            s = sfa1.actD[a1]
            if contain(s,voc2,set()):
                for r1 in sfa1.trans[q[0]][a1]:
                    r = (r1,q[1])
                    morep.add(r)
                    temp.actD[s2s(s)] = s
                    add2stateD(temp,r)
                    qcode = decodeState(temp,q)
                    di = {s2s(s): {decodeState(temp,r)}} 
                    if qcode in temp.trans:
                         temp.trans[qcode] = dicMerge(temp.trans[qcode],
                                                      di)
                    else:
                         temp.trans[qcode] = di
    return morep

# To add d2 to p [side-effects on temp]
def ruled2(sfa2,voc1,q,temp): 
    morep = set()
    moretrans = {}
    if q[1] in sfa2.trans:
        for a2 in sfa2.trans[q[1]]:
            s = sfa2.actD[a2]
            if contain(s,voc1,set()):
                for r2 in sfa2.trans[q[1]][a2]:
                    r = (q[0],r2)
                    morep.add(r)
                    temp.actD[s2s(s)] = s
                    add2stateD(temp,r)
                    qcode = decodeState(temp,q)
                    di = {s2s(s): {decodeState(temp,r)}} 
                    if qcode in temp.trans:
                         temp.trans[qcode] = dicMerge(temp.trans[qcode],
                                                      di)
                    else:
                         temp.trans[qcode] = di
    return morep

# To add visited states to statedD.
def add2stateD(sfa,r):
    old = False
    for i in sfa.stateD:
        if sfa.stateD[i]=={1:r[0], 2:r[1]}:
            old = True
    if old == False:
        sfa.stateD[len(sfa.stateD)] = {1:r[0], 2:r[1]}

# To decode state values.
def decodeState(sfa,r):
    found = -1
    for i in sfa.stateD:
        if sfa.stateD[i]=={1:r[0], 2:r[1]}:
            found = i
    return found

# To check for a subset of a set.
def contain(set1,set2,set3):
    return set1.intersection(set2).issubset(set3)


'''
PART 1B:
Function definitions to Superpose greater than or equal to 2 automaton of s-words.
'''
# To define the main calling function.
def supULD(n):
    temp = uld(n).make_sfa(n)
    if n == 1:
        return temp
    elif 1<n:
        return sup(temp,supULD(n-1))

# To get all the vocabularies based on the given noOfEvents >= 1. 
def voc(n):
    return {n,-n}

# Recursive function definition to get the union of all vocabularies based on noOfEvents >= 1.
def vocA(n):
    temp = voc(n)
    if n == 1:
        return temp
    elif 1<n:
        return temp.union(vocA(n-1))   
    
'''
PART 2:
Function definitions to Projection via Allen-composition
'''

def even_sfa(sfa0): # rename state q to 2*q+2 (saving 0 for later) 
    temp = sfa()
    temp.init = 2*sfa0.init+2
    for q in sfa0.final:
        temp.final.add(2*q+2)
    for q in sfa0.trans:
        temp.trans[2*q+2] = even_tr(sfa0,q)
    for q in sfa0.stateD:
        temp.stateD[2*q+2] = sfa0.stateD[q]
    temp.actD = sfa0.actD
    temp.labels = sfa0.labels
    return temp

def even_tr(sfa0,q): # rename transitions for even states
    tr = {}
    for a in sfa0.trans[q]:
        tr[a] = set()               
        for qq in sfa0.trans[q][a]:
            tr[a].add(2*qq+2)
    return tr

def odd_sfa(sfa0): # rename state q to 2*q+1
    temp = sfa()
    temp.init = 2*sfa0.init+1
    for q in sfa0.final:
        temp.final.add(2*q+1)
    for q in sfa0.trans:
        temp.trans[2*q+1] = odd_tr(sfa0,q)
    for q in sfa0.stateD:
        temp.stateD[2*q+1] = sfa0.stateD[q]
    temp.actD = sfa0.actD
    temp.labels = sfa0.labels
    return temp

def odd_tr(sfa0,q): # rename transitions  for odd states
    tr = {}
    for a in sfa0.trans[q]:
        tr[a] = set()               
        for qq in sfa0.trans[q][a]:
            tr[a].add(2*qq+1)
    return tr

def sfaUnion(sfa1,sfa2): # disjoint union using even_sfa, odd_sfa
    temp = even_sfa(sfa1)
    n2 = odd_sfa(sfa2)
    temp.trans = transMerge(temp.trans,n2.trans)
    temp.stateD = dicUnion(temp.stateD,n2.stateD)
    temp.trans[0] = dicUnion0(temp.trans[temp.init],
                              n2.trans[n2.init])
    temp.stateD[0] = dicUnion0(temp.stateD[temp.init],
                               n2.stateD[n2.init])
    temp.init = 0
    temp.final.update(n2.final)
    temp.actD = dicUnion0(temp.actD,n2.actD)
    temp.labels.update(n2.labels)
    return temp

def transMerge(dic1,dic2):
    temp = dic1.copy()
    for key in dic2:
        if key in dic1:
            temp[key] = dicUnion(dic1[key],dic2[key])
        else:
            temp[key] = dic2[key]
    return temp

def project(voc0,sfa0):  # voc0-depad and then depad
    return depad(reduct(voc0,sfa0))
    
def reduct(voc0,sfa0):   # voc0-reduct
    temp = sfa()
    for a in sfa0.actD:
        ar = sfa0.actD[a].intersection(voc0)
        temp.actD[s2s(ar)] = ar        
    for i in sfa0.trans:
        temp.trans[i] = newTrans(sfa0.trans[i],voc0,sfa0.actD)
    temp.stateD = sfa0.stateD
    temp.labels = sfa0.labels
    temp.final = sfa0.final
    return temp

def newTrans(trans0,voc0,actD0):
    temp = {}
    for a in trans0:
        ar = actD0[a].intersection(voc0)
        sar = s2s(ar)
        if sar in temp:
            temp[sar].update(trans0[a])
        else:
            temp[sar] = trans0[a]
    return temp


def depad(sfa0):  # depad:  e-close then e-remove
    if str([]) in sfa0.actD:  # s2s(set()) = str([])
        e_close(sfa0.trans)
        e_remove(sfa0.actD,sfa0.trans)
# consider trimming temp.stateD and temp.labels
    return sfa0
 
#  e_close: extend trans minimally to trans2 s.t.
#      for all states q1,q2 s.t. q2 in trans2[q1][set()]
#            (1) whenever trans2[q][a] has q1, it also has q2
#        and (2) whenever trans2[q2][a] has q, trans2[q1][a] also has q
#
#   can extend at most len(stateD) rounds
#
def e_close(trans):
    step = True
    while step == True:
        step = e_step(trans)

def e_step(trans):
    more = False    
    arc = e_arc(trans)
    for p in arc:
        for q in trans:
            for a in trans[q]:
                if p[0] in trans[q][a] and not(p[1] in trans[q][a]):
                    trans[q][a].add(p[1])
                    more = True
        if p[1] in trans:
            for a in trans[p[1]]:
                for q in trans[p[1]][a]:
                    if a in trans[p[0]]:
                        if not(q in trans[p[0]][a]):
                            trans[p[0]][a].add(q)
                            more = True
                    else:
                        trans[p[0]][a] = {q}
                        more = True
    return more

def e_arc(trans):
    temp = []
    eps = str([])   # = s2s(set()))
    for q1 in trans:
        if eps in trans[q1]:
            for q2 in trans[q1][eps]:
                if not((q1,q2) in temp):
                    temp.append((q1,q2))
    return temp
                                     
#  e_remove  s2s(set()) from sfa.actD.values() and
#                            from sfa.trans[q]  for all states q
def e_remove(actD,trans):
    eps = str([])      # = s2s(set()))                                
    actD.pop(eps)
    for q in trans:
        if eps in trans[q]:
            trans[q].pop(eps)



# consider simplifying xsfa to 
# def s2sfa(input0,sfa0): # string to sfa, but what is sfa0.labels?
        
def xsfa(sfa0,input0): # sfa returned may have states outside range(leng(stateD)
    run = sfa0.trace(sfaInput(input0))
    temp = sfa()
    q = sfa0.init
    temp.init = q
    temp.stateD[q] = sfa0.stateD[q]
    temp.labels = getK(sfa0.stateD[q])
    if run == "rejected":
        print("Nothing to extract here")
    else:    
        for i in run:
            temp.trans[q] = { i[0]: {i[1]} }
            temp.actD[i[0]] = sfa0.actD[i[0]]
            q = i[1]
            temp.stateD[q] = sfa0.stateD[q]
            temp.labels.update(getK(sfa0.stateD[q]))
        temp.final = {q}
    return temp

def getK(dic):  # returns set of dic keys
    temp = set()
    for i in dic:
        temp.add(i)
    return temp


def alle(a,b):
    return sup(uld(a).make_sfa(a),uld(b).make_sfa(b))

def als(reln,a,b): # returns string for allen reln on a,b
    if reln == 'e':
        return [{a,b},{-a,-b}]
    elif reln == 'p':
        return [{a},{-a},{b},{-b}]
    elif reln == 'm':
        return [{a},{b,-a},{-b}]
    elif reln == 'o':
        return [{a},{b},{-a},{-b}]
    elif reln == 'F':
        return [{a},{b},{-a,-b}]
    elif reln == 'D':
        return [{a},{b},{-b},{-a}]
    elif reln == 's':
        return [{a,b},{-a},{-b}]
    elif reln == 'S':
        return [{a,b},{-b},{-a}]
    elif reln == 'd':
        return [{b},{a},{-a},{-b}]
    elif reln == 'f':
        return [{b},{a},{-a,-b}]
    elif reln == 'O':
        return [{b},{a},{-b},{-a}]
    elif reln == 'M':
        return [{b},{-b,a},{-a}]
    elif reln == 'P':
        return [{b},{-b},{a},{-a}]

def allen1(reln,a,b): # returns sfa for allen reln  # vary a,b for allen compos
    f = alle(a,b)
    return xsfa(f,als(reln,a,b))
    
def allenL(relnList,a,b): # return sfa accepting allen relns in relnList
    temp = allen1(relnList[0],a,b)
    for r in relnList[1:]:
        temp = sfaUnion(temp,allen1(r,a,b))
    return temp

def allenc(rel1,rel2,a,b,c): # return sfa accepting any allen reln in comp
    sfa1 = allen1(rel1,a,b)
    sfa2 = allen1(rel2,b,c)
    sfa0 = sup(sfa1,sfa2)
    return project({a,c,-a,-c},sfa0)

def whichAR(sfa,a,b):  # returns allen relations on a,b accepted by sfa
    tempRel = []
    tempPath = []
    ar = ['p', 'm', 'o', 'F', 'D', 's',
          'P', 'M', 'O', 'f', 'd', 'S', 'e']
    for r in ar:
        if sfa.acc(als(r,a,b)) == "accepted":
            tempRel.append(r)
            tempPath.append(als(r,a,b))
    return tempRel,tempPath

'''
VISUALIZATION PART  1A, 1B, 2:
Initializing functions responsible for visualization.
'''

# To construct a dictionary to store actions and destinations possible for all possible state in the superposed automaton.
def transition(dict1,dict2,end):
    x = end.pop()
    tempDict = dict1.copy()
    for i in dict1.keys():
        for j in dict2.keys():
            if i == j:
                tempDict[str(dict2[j])] = tempDict[i]
                del tempDict[i]
    tempDict[str(dict2[x])] = {'.' : {str(dict2[x])}}
    return tempDict

# To construct a dictionary to store actions and destinations possible for all possible state in the sWord automaton.
def transitionSwords(dict1,dict2,end):
    x = end.pop()
    tempDict = dict1.copy()
    visited = set()
    for i in dict1.keys():
        for j in dict2.keys():
            if j not in visited:
                visited.add(j)
                tempDict[str(dict2[j])] = tempDict[i]
                del tempDict[i]
                break
    tempDict[str(dict2[x])] = {'[ ]' : {str(dict2[x])}}
    return tempDict

# To construct a set consisting of all possible actions within the automaton defined.
def actions(dict):
    temp = set()
    for i in dict.keys():
        for j in dict[i].keys():
            if j not in temp:
                temp.add(j)
    return temp

# To construct a set consisting of all possible states within the automaton defined.
def state(dict):
    temp = set()
    for i in dict.keys():
        temp.add(i)
    return temp

# To replace special character state representatio of ':' to '-'
def convertStateD(dict):
    temp = ()
    dtemp = {}
    for i,j in dict.items():
        temp = str(j)
        dtemp[i] = temp.replace(":", '-')
    return dtemp

# To change state representation of <action:State> pair within the superposed tranState dictionary. 
def finalTransition(dict1, dict2):
    tempDict = copy.deepcopy(dict1)
    for i in dict1.keys():
        for j in dict1[i].keys():
            for k in dict2.keys():
                for l in dict1[i][j]:
                    if l == k:
                        #dict1[i][j] = {str(dict2[k])}
                        tempDict[i][j] = {str(dict2[k])}
                        break
    return tempDict

# To change state representation of <action:State> pair within the sWord transtate dictionary. 
def finalTransitionSwords(dict1, dict2):
    visited = set()
    l = 1
    tempDict = copy.deepcopy(dict1)
    for i in dict1.keys():
        for j in dict1[i].keys():
            for k in list(dict2.values())[l:]:
                if k not in visited:
                    visited.add(k)
                    tempDict[i]['[ ]'] = listify(i)
                    tempDict[i][j] = listify(k)
                    l=l+1
                    break
            break
    return tempDict

# To get the inital state of the automaton
def initial(dict):
    temp = str(dict[0])
    return temp

# get the terminal state of the automaton.
def finale(dict,end):
    temp = str(dict[end.pop()])
    return set(temp)

# To delete the terminal state.
def removeState(dict1,dict2,end):
    x = end.pop()
    temp = copy.deepcopy(dict1)
    del temp[str(dict2[x])]
    return temp


'''
PART 1A:
Graphical user interface build upon superposing functionalities.
'''

# To clear frame canvas.
def clearCanvas(frame):
    for widget in frame.winfo_children():
        widget.destroy()

# To display states and transitions for event 1. 
def showAutomaton1(root):
    clearCanvas(root)
    n = eventNumber1.get()
    Label(root, text = "ULD for interval 1 from uld(1):").place(x = 0, y = 25)  #label
    Label(root, text = "Here u: Unborn l: Living & d:Dead").place(x = 0, y = 50)
    fa1 = uld(n)
    Label(root, text = "Inital Sate:").place(x = 0, y = 90)
    Label(root, text = fa1.init).place(x = 100, y = 90)  #label 
    Label(root, text = "Final Sate:").place(x = 0, y = 110)
    Label(root, text = fa1.final).place(x=100, y=110)
    Label(root, text = "Transitions:").place(x = 0, y = 130)
    Label(root, text = fa1.trans).place(x=100, y=130)
    Wdisplay1 = Button(root, text="Display the Sword automaton", command= lambda: displayAutomaton1(fr, fa1)).place(x=0, y=150)


# To display automaton for event 1. 
def displayAutomaton1(root, fa1):
    global img1
    n = eventNumber1.get()
    f1 = fa1.make_sfa(n)
    temp = copy.deepcopy(f1.trans)
    end = f1.final.copy()
    end1 = f1.final.copy()
    node = convertStateD(f1.stateD) 
    tranState = transitionSwords(temp,node,end)
    nf_1 = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState),
    transitions=finalTransitionSwords(tranState,node),
    initial_state=node[0],
    final_states={node[2]},
    )
    nf1 = VisualNFA(nf_1)
    nf1.show_diagram().render("automaton1", format='jpg')
    Label(root, text = "The Transition Diagram", font="Calibri 15").place(x=0, y=175)
    try:
        image = Image.open("C:/Users/joshu/automaton1.jpg")
        image = image.resize((500, 150), Image.ANTIALIAS)
        img1 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img1).place(x=10, y=205)
    panel.pack()


# To Display states and transitions for event 2. 
def showAutomaton2(root):
    n = eventNumber2.get()
    Label(root, text = "ULD for interval 2 from uld(2):").place(x = 600, y = 25)  #label
    Label(root, text = "Here u: Unborn l: Living & d:Dead").place(x = 600, y = 50)
    fa2 = uld(n)
    Label(root, text = "Inital Sate:").place(x = 600, y = 90)
    Label(root, text = fa2.init).place(x = 700, y = 90)  #label 
    Label(root, text = "Final Sate:").place(x = 600, y = 110)
    Label(root, text = fa2.final).place(x=700, y=110)
    Label(root, text = "Transitions:").place(x = 600, y = 130)
    Label(root, text = fa2.trans).place(x=700, y=130)
    Wdisplay1 = Button(root, text="Display the Sword automaton", command= lambda: displayAutomaton2(fr, fa2)).place(x=600, y=150)
    


# To display automaton for event 2. 
def displayAutomaton2(root, fa2):
    global img2
    n = eventNumber2.get()
    f2 = fa2.make_sfa(n)
    temp = copy.deepcopy(f2.trans)
    end = f2.final.copy()
    end1 = f2.final.copy()
    node = convertStateD(f2.stateD) 
    tranState = transitionSwords(temp,node,end)
    nf_2 = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState),
    transitions=finalTransitionSwords(tranState,node),
    initial_state=node[0],
    final_states={node[2]},
    )
    nf2 = VisualNFA(nf_2)
    nf2.show_diagram().render("automaton2", format='jpg')
    Label(root, text = "The Transition Diagram", font="Calibri 15").place(x=600, y=175)
    try:
        image = Image.open("C:/Users/joshu/automaton2.jpg")
        image = image.resize((500, 150), Image.ANTIALIAS)
        img2 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img2).place(x=600, y=205)
    panel.pack()


# To display superposition automaton for 2 events. 
def displaySuperposition(root):
    global img
    n1 = eventNumber1.get()
    n2 = eventNumber2.get()
    fa1 = uld(n1)
    f1 = fa1.make_sfa(n1) 
    fa2 = uld(n2)
    f2 = fa2.make_sfa(n2) 
    fa = sup(f1,f2)
    temp = copy.deepcopy(fa.trans)
    end = fa.final.copy()
    end1 = fa.final.copy()
    l = fa.final.copy()
    x = l.pop()
    node = convertStateD(fa.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    nfa = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState1),
    transitions=finalTransition(tranState1,node),
    initial_state=node[0],
    final_states={node[x]},
    )
    nfn = VisualNFA(nfa)
    nfn.show_diagram().render("superpose2automaton", format='jpg')
    Label(root, text = "The Transition Diagram").place(x=0, y=0)
    try:
        image = Image.open("C:/Users/joshu/superpose2automaton.jpg")
        image = image.resize((800, 350), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img).place(x=10, y=20)
    panel.pack()


# To Check states accepted in automaton or not. 
def acceptanceStateCheck(root):
    temp = []
    n1 = eventNumber1.get()
    n2 = eventNumber2.get()
    f1 = uld(n1)
    f1 = f1.make_sfa(n1) 
    f2 = uld(n2)
    f2 = f2.make_sfa(n2) 
    fa = sup(f1,f2)
    temp.extend(transState.get().split("."))
    ans = fa.mapper(temp)
    Label(root, text = ans, font="Calibri 10").place(x=200, y=610)

'''
PART 1B:
Superposing Multiple Events. 
'''

# To define the User Interface window for superposing more than 1 events. 
def showSuperposeIntervalWindow(root):
    newWindow1 = Toplevel(root)
    newWindow1.title("Automaton for more than two events")
    newWindow1.geometry("1500x1000+30+30")
    Label(newWindow1, text = "ENTER THE TOTAL NUMBER OF EVENTS:[GREATER THAN 1]", font="Calibri 15").place(x=5, y=10)
    Entry(newWindow1, textvariable = noOfEvents, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 40)#entry textbox
    Label(newWindow1, text = "ENTER THE START STATE:", font="Calibri 15").place(x=5, y=80)
    Entry(newWindow1, textvariable = startState , bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 110)#entry textbox
    button1 = Button(newWindow1, text="Dispay Automaton", command= lambda: displaySuperposeIntervals(newWindow1)).place(x=5, y = 140)
    Label(newWindow1, text = "ENTER THE TRANSITION PATH", font="Calibri 15").place(x=700, y=10)
    Entry(newWindow1, textvariable = transPath, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=700, y = 40)#entry textbox
    button2 = Button(newWindow1, text="Dispay Acceptance", command= lambda: displayPathAcceptance(newWindow1)).place(x=700, y = 60)
    button3 = Button(newWindow1, text="Projection Window", command= lambda: showProjectionsWindow(root)).place(x=650, y = 775)

# To convert the pdf into jpg format 
def pdfConvertor():
    pages = convert_from_path('C:/Users/joshu/AllenSuperposition.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/AllenSuperposition.jpg', 'JPEG')
        
# To construct a subset of States and its associated <action:childState> pair.
def subState(dict1,currentState, tempStorage):
    for i in currentState:            
        if str(i) in tempStorage.keys():
            currentState.remove(i)
            continue
        else:  
            try:
                tempStorage[str(i)] = dict1[str(i)]
                for j in dict1[str(i)].values():
                    currentState.extend(j)
            except:
                currentState.remove(i)
                break
    if len(currentState) != 0:
        subState(dict1,currentState, tempStorage)
    return tempStorage    
    
# Display superposed intervals. 
def displaySuperposeIntervals(root):                     
    global img3  
    tempStorage = {}
    n = noOfEvents.get()
    curNode = startState.get()
    s1 = listify(curNode)
    fn = supULD(n) 
    temp = copy.deepcopy(fn.trans)
    end = fn.final.copy()
    end1 = fn.final.copy()
    l = fn.final.copy()
    x = l.pop()
    node = convertStateD(fn.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    temp1 = subState(finalTransition(tranState1,node), s1, tempStorage)
    allStates = state(temp1)
    allStates.add(node[x])
    nf_n = VisualNFA(
    states= allStates,
    input_symbols=actions(tranState1),
    transitions=temp1,
    initial_state=curNode,
    final_states={node[x]},
    )
    nfn = VisualNFA(nf_n)
    nfn.show_diagram().render("superposeNautomaton", format='jpg')
    Label(root, text = "The Transition Diagram").place(x=5, y=170)
    try:
        image = Image.open("C:/Users/joshu/superposeNautomaton.jpg")
        image = image.resize((600, 600), Image.ANTIALIAS)
        img3 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img3).place(x=10, y=200)
    panel.pack()

# To highlight the accepted path over superposed intervals. 
def displayPathAcceptance(root):
    global img4 
    n = noOfEvents.get()
    tempPath = []
    fn = supULD(n) 
    temp = copy.deepcopy(fn.trans)
    end = fn.final.copy()
    end1 = fn.final.copy()
    l = fn.final.copy()
    x = l.pop()
    tempPath.extend(transPath.get().split("."))
    node = convertStateD(fn.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    nf_n = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState1),
    transitions=finalTransition(tranState1,node),
    initial_state=node[0],
    final_states={node[x]},
    )
    nfn = VisualNFA(nf_n)
    nfn.show_diagram(tempPath).render("pathAcceptance", format='jpg')
    Label(root, text = "The Acceptance Path Diagram").place(x=700, y=90)
    try:
        image = Image.open("C:/Users/joshu/pathAcceptance.jpg")
        image = image.resize((600, 600), Image.ANTIALIAS)
        img4 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img4).place(x=750, y=150)
    panel.pack()


'''
PART 2:
Projection via Allen Composition. 
''' 

# To define the User Interface window for Projections. 
def showProjectionsWindow(root):
    newWindow2 = Toplevel(root)
    newWindow2.title("Projection via Allen Composition")
    newWindow2.geometry("1500x1000+30+30")
    Label(newWindow2, text = "ENTER THE FIRST ALLEN RELATION BETWEEN EVENT 1&2", font="Calibri 15").place(x=5, y=10)
    Entry(newWindow2, textvariable = allenFirstRel, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 40)#entry textbox
    button1 = Button(newWindow2, text="Dispay Automaton", command= lambda: displayAllenFirstRel(newWindow2)).place(x=5, y = 60)
    Label(newWindow2, text = "ENTER THE SECOND ALLEN RELATION BETWEEN EVENT 2&3", font="Calibri 15").place(x=700, y=10)
    Entry(newWindow2, textvariable = allenSecondRel, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=700, y = 40)#entry textbox
    button2 = Button(newWindow2, text="Display Automaton", command= lambda: displayAllenSecondRel(newWindow2)).place(x=700, y = 60)
    button3 = Button(newWindow2, text="Display Superposition Automaton", command= lambda: displayAllenSuperpositions(newWindow2)).place(x=5, y = 310)
    fr = Frame(newWindow2, height=100, width=1500, highlightbackground="blue", highlightthickness=1)
    fr.pack_propagate(0) # don't shrink
    fr.place(x=10, y=680)
    Label(fr, text = "ENTER THE TWO EVENTS WHOSE PROJECTION TO BE EXTRACTED:", font="Calibri 15").place(x=5, y=2)
    Entry(fr, textvariable = projectionReq, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 30)#entry textbox
    button4 = Button(fr, text="Display Possible Allen Relations", command= lambda: displayProjections(fr)).place(x=5, y = 55)
    button5 = Button(newWindow2, text="Calculate Conditional Probability", command= lambda: calculateProbabilityWindow(root)).place(x=650, y = 800)
    
# To display automaton based on allen relation between event 1 and 2.
def displayAllenFirstRel(root):
    global img_a1
    rel = allenFirstRel.get()
    a1 = allen1(rel,1,2)
    temp = copy.deepcopy(a1.trans)
    end = a1.final.copy()
    end1 = a1.final.copy()
    l = a1.final.copy()
    x = l.pop()
    node = convertStateD(a1.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    nfa = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState1),
    transitions=finalTransition(tranState1,node),
    initial_state=node[0],
    final_states={node[x]},
    )
    nfn = VisualNFA(nfa)
    nfn.show_diagram().render("AllenFirstRel", format='jpg')
    Label(root, text = "The Transition Diagram").place(x=10, y=95)
    try:
        image = Image.open("C:/Users/joshu/AllenFirstRel.jpg")
        image = image.resize((650, 175), Image.ANTIALIAS)
        img_a1 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img_a1).place(x=10, y=125)
    panel.pack()    

# To display automaton based on allen relation between event 2 and 3.
def displayAllenSecondRel(root):
    global img_a2
    rel = allenSecondRel.get()
    a2 = allen1(rel,2,3)
    temp = copy.deepcopy(a2.trans)
    end = a2.final.copy()
    end1 = a2.final.copy()
    l = a2.final.copy()
    x = l.pop()
    node = convertStateD(a2.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    nfa = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState1),
    transitions=finalTransition(tranState1,node),
    initial_state=node[0],
    final_states={node[x]},
    )
    nfn = VisualNFA(nfa)
    nfn.show_diagram().render("AllenSecondRel", format='jpg')
    Label(root, text = "The Transition Diagram").place(x=700, y=95)
    try:
        image = Image.open("C:/Users/joshu/AllenSecondRel.jpg")
        image = image.resize((750, 175), Image.ANTIALIAS)
        img_a2 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img_a2).place(x=700, y=125)
    panel.pack() 

# To display superposed allen automaton outcome between relation 1 and relation 2.
def displayAllenSuperpositions(root):
    global img_aS
    rel1 = allenFirstRel.get()
    a1 = allen1(rel1,1,2)
    rel2 = allenSecondRel.get()
    a2 = allen1(rel2,2,3)
    aS = sup(a1,a2)
    temp = copy.deepcopy(aS.trans)
    end = aS.final.copy()
    end1 = aS.final.copy()
    l = aS.final.copy()
    x = l.pop()
    node = convertStateD(aS.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    nfa = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState1),
    transitions=finalTransition(tranState1,node),
    initial_state=node[0],
    final_states={node[x]},
    )
    nfn = VisualNFA(nfa)
    nfn.show_diagram().render("AllenSuperposition", format='pdf', view = True)
    pdfConvertor()
    Label(root, text = "The Transition Diagram").place(x=250, y=330)
    try:
        image = Image.open("C:/Users/joshu/AllenSuperposition.jpg")
        image = image.resize((850, 325), Image.ANTIALIAS)
        img_aS = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(root, image = img_aS).place(x=250, y=350)
    panel.pack()
    
# To get all possible allen relation from the superposed automaton based on event 1&2 or 1&3 or 2&3. 
def displayProjections(root):
    global tracker
    fr1 = Frame(root, height=88, width=475, highlightbackground="blue", highlightthickness=1)
    fr1.pack_propagate(0) # don't shrink
    fr1.place(x=690, y=5) 
    fr = Frame(root, height=80, width=200, highlightbackground="blue", highlightthickness=1)
    fr.pack_propagate(0) # don't shrink
    fr.place(x=700, y=15)
    rel1 = allenFirstRel.get()
    a1 = allen1(rel1,1,2)
    rel2 = allenSecondRel.get()
    a2 = allen1(rel2,2,3)
    req = projectionReq.get()
    try: 
        if tracker == True:
            clearCanvas(fr1)
    except:
        pass
    if req == "1 2":
        comp = project({1,2,-1,-2},sup(a1,a2))
        resRel,resPath = whichAR(comp,1,2)
    elif req == "1 3":
        comp = project({1,3,-1,-3},sup(a1,a2))
        resRel,resPath = whichAR(comp,1,3)
    else:
        comp = project({2,3,-2,-3},sup(a1,a2))
        resRel,resPath = whichAR(comp,2,3) 
    clearCanvas(fr)
    for i in range(len(resRel)):
        for j in range(2):
            e = Entry(fr, width=30, fg='blue', font=('Arial', 10, 'bold'))
            e.grid(row=i, column=j)
            if j == 0:
                e.insert(END, resRel[i])
            else:
                e.insert(END, resPath[i])
    tracker = True
    
    
'''
SECTION 2:
1. Conditional Probability Calculations.
'''
'''
PART 3:
Function definitions to construct [Uborn,Living,Died] Non-Deterministic Finite automaton for 'n' events/Intervals.
'''

# Definning Final and Initial State based on Number of Intervals.
def finalState(n):
    temp = [] #Temporary Storage list to hold final state of an automaton.
    for i in range(n):
        temp += [2] #Appending '2' for each intervals signifying the event is died/finished.
    return temp

# Definning Previous State based on Current State. [Making use of Recursion function]
def prevState(n,q):  # n=number of intervals, q=string of length n over [0,1,2]
    temp = []
    if q[0] == 1:
        temp.append([0]+q[1:])
    elif q[0] == 2:
        temp.append([1]+q[1:])
    if n > 1:
        for c in prevState(n-1,q[1:]):  # Recursive Function to determine all the previous state for a given/current state.
            temp.append(q[0:1]+c)
            if q[0] == 1:
                temp.append([0]+c)
            elif q[0] == 2:
                temp.append([1]+c)
    return temp

# To update total number of strings between current and final node, as well as all the child node for the given current node.
def upd(dict,p,q):
    if str(p) in dict.keys():
        if q in dict[str(p)]:
            return dict[str(p)]
        else: 
            val = dict[str(p)][0] + dict[str(q)][0] 
            lis = [val,q] + dict[str(p)][1:]
            return lis
    else:
        return(dict[str(q)][0:1] + [q])

# To get only the total number of strings between current and final node
def clean(dict):
    temp = {}
    for i in dict:
        temp[i] = dict[i][0]
    return temp

# To calculate the conditional probability for between two adjacent states/events in the automaton.
def transPr(q0,q1,n):
    temp = count(n)
    if q1 in str(temp[q0]):
        c0 = temp[q0][0]
        c1 = temp[q1][0]
        return(str(c1) + "/" + str(c0))
    else:
        return(0)

# Definning dictionary to store State as key and the number of child nodes and child state as value. 
def count(n):   # dictionary  state:[number,next...next']
    global fstate,istate,tstate
    tstate = str(finalState(n))
    fstate = str(finalState(n)) +" "+'1' 
    temp = { str(finalState(n)) : [1] }
    back = [finalState(n)]
    for i in range(2*n):# Number of actions possible for a given interval/event 'n'
        new = []
        for state in back:
            more = prevState(n,state)
            for pstate in more:
                temp.update({ str(pstate): upd(temp,pstate,state) })    
            new.extend(more)
        back = new
    return temp

'''
VISUALIZATION PART 3:
Initializing functions responsible for visualization.
'''    
# To construct a subset of States and its associated <action:childState> pair.
def lowerState(dict,currentState, tempStorage):
    for i in currentState:            
        if str(i) in tempStorage.keys():
            currentState.remove(i)
            continue
        else:
            tempStorage[str(i)] = dict[str(i)]
            currentState.extend(dict[str(i)][1:])
    if len(currentState) != 0:
        lowerState(dict,currentState, tempStorage)
    return tempStorage

#  To extract inital state of an automaton for the given noOfEvents.
def initState(x):
    temp = x[0]
    return temp

# To extract all possible state of an automaton for the given noOfEvents.
def possibleState(dict):
    temp = [] 
    for i in dict.keys():
        temp += [i]
    return temp

# To modify the extracted possible state of an automaton for the given noOfEvents.
def allState(dict):
    temp = [] 
    for i in dict.keys():
        temp += [i +" " + str(dict[i])]
    return temp

# To construct a dictionary to store actions and destinations possible for all possible state in the automaton 
# for the given noOfEvents.
def transitionULD(dict):
    temp = {}
    for i in dict:
        trans = set()
        if i == tstate:
            continue;
            #temp[i + " " + '1'] = {"1": {str(tstate) +" "+"1"}}
        else: 
            for s in dict[i][1:]:
                trans.add(str(s) + " " + str(dict[str(s)][0]))
            temp[i + " " + str(dict[i][0])] = {".": trans}
    return temp

# To Convert the pdf into jpg format. 
def pdfConvertor():
    pages = convert_from_path('C:/Users/joshu/Digraph.gv.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/automaton.jpg', 'JPEG')
        
# To Retieve & Display the possible states for the given number of events.
def retrieveStates(root):   
    clearCanvas(root)
    Label(root, text = "The possible states are:").place(x = 0, y = 25)  #label
    Label(root, text = "Here 0: Unborn 1: Living & 2:Dead").place(x = 0, y = 50) 
    n = totalEvents.get()
    z = 0
    j = 0
    for i,s in enumerate(possibleState(clean(count(n)))):
        f = Frame(root, height=20, width=100)
        f.pack_propagate(0) # don't shrink
        if (i+1)%10 == 0:
            z = z + 25
            j = 0
        j = j + 1
        f.place(x=100+((j)*100), y= z + 100)       
        label = Label(f, text = s, borderwidth = 1, relief = "solid", bg = 'linen')
        label.pack(fill=BOTH, expand=True)


# To retieve & Display the conditional Probability between the given two states: 
def retrieveProbability(root):   
    clearCanvas(root)
    state1 = childState.get()
    state2 = parentState.get()
    n = totalEvents.get()
    ans = transPr(state2,state1,n)
    breakValue.set("The Conditional probability between {} and {} is given by:".format(state1,state2))
    Label(root, textvariable = breakValue).place(x = 0, y = 25)  #label 
    Label(root, text = ans, font="Calibri 20").place(x=5, y=50)

'''
Graphical user interface build to demonstrate Conditional Probability.
'''
# To define a new window for demonstrating probability calculations.
def calculateProbabilityWindow(root):
    newWindow3 = Toplevel(root)
    newWindow3.geometry("1500x1000+30+30")
    newWindow3.title("Conditional Probability Calculation for a given number of events in {U,L,D}")
    canvas = Canvas(newWindow3)
    Label(newWindow3, text="ENTER THE TOTAL NUMBER OF EVENTS:", font="Calibri 12").place(x=10, y=0)  #label
    Label(newWindow3, text="No of Events[Any Integer value greater than 1]").grid(row = 0, padx=(10, 10), pady = (25,0))  #label
    Entry(newWindow3, textvariable = totalEvents, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=0, column=3, pady = (25,0)) #entry textbox

    fr = Frame(newWindow3, height=400, width=1350, highlightbackground="blue", highlightthickness=1)
    fr.pack_propagate(0) # don't shrink
    fr.place(x=75, y=80)
    button1 = Button(newWindow3, text="Get the list of states in the automaton", command= lambda: retrieveStates(fr)).grid(row=3, column=0, padx=(10, 10), sticky=W)
    button2 = Button(newWindow3, text="Clear", command= lambda: clearCanvas(fr)).grid(row=4, column=0, padx=(10, 10),  sticky=W)
    Label(newWindow3, text="ENTER TWO CONSECUTIVE STATES TO CALCULATE THE CONDITIONAL PROBABILITY BETWEEN THEM:", font="Calibri 12").place(x=10, y=500)  #label
    fr1 = Frame(newWindow3, height=150, width=1350, highlightbackground="blue", highlightthickness=1)
    fr1.pack_propagate(0) # don't shrink
    fr1.place(x=75, y=630)
    Label(newWindow3, text="Parent State").place(x = 10, y = 525)   #label
    Entry(newWindow3, textvariable = parentState, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=175, y = 525)#entry textbox
    button3 = Button(newWindow3, text="Display Automaton", command= lambda: drawAutomaton2(newWindow3)).place(x=375, y = 525)
    Label(newWindow3, text="Child State").place(x=10, y=555)   #label
    Entry(newWindow3, textvariable = childState, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=175, y = 555)
    button4 = Button(newWindow3, text="Display Automaton", command= lambda: drawAutomaton1(newWindow3)).place(x=375, y = 555)
    button5 = Button(newWindow3, text="Get the conditional Probability", command = lambda: [retrieveProbability(fr1)]).place(x=10, y = 585)
    button6 = Button(newWindow3, text="Clear", command= lambda: clearCanvas(fr1)).place(x=10, y = 610)
    button7 = Button(newWindow3, text="Gap Implementation", command= lambda: gapImplementationWindow(root)).place(x=650, y = 800)

# To define and display a part of automaton of total event size = 'n'
def drawAutomaton1(root):
    global img_cs
    tempStorage = {}
    curNode = childState.get()
    s1 = listify(curNode)
    n = totalEvents.get()
    fState = str(finalState(n))
    vv = lowerState(count(n),s1, tempStorage)
    if curNode == fState:
        gra = Digraph(format = 'jpg')
        gra.node('a', fState ,shape='doublecircle')
        newWindow = Toplevel(root)
        newWindow.title("Automaton for" + " "+ childState.get())
        newWindow.geometry("1500x1000+30+30")
        Label(newWindow, text = "The Transition Diagram", font="Calibri 15").place(x=5, y=250)
        gra.render('fState', format = 'jpg' )
        try:
            image = Image.open("C:/Users/joshu/fState.jpg")
            image = image.resize((400, 350), Image.ANTIALIAS)
            img_cs = ImageTk.PhotoImage(image)
        except:
            print('Issue in drawing automaton')
    else:    
        nfa1 = VisualNFA(
        states= set(allState(clean(vv))),
        input_symbols={'.'},
        transitions=transitionULD(vv),
        initial_state=initState(allState(clean(vv))),
        final_states={fstate},
        )
        nfa1 = VisualNFA(nfa1)
        nfa1.show_diagram().render("subAutomaton1", format='jpg')
        pdfConvertor()
        newWindow = Toplevel(root)
        newWindow.title("Automaton for" + " "+ childState.get())
        newWindow.geometry("1500x1000+30+30")
        Label(newWindow, text = "The Transition Table:", font="Calibri 15").place(x=5, y=50)
        Label(newWindow, text = nfa1.table, font="Calibri 10").place(x=5, y=125)
        Label(newWindow, text = "The Transition Diagram", font="Calibri 15").place(x=5, y=250)
        try:
            image = Image.open("C:/Users/joshu/subAutomaton1.jpg")
            image = image.resize((500, 450), Image.ANTIALIAS)
            img_cs = ImageTk.PhotoImage(image)
        except:
            print('Issue in drawing automaton')
    panel = tk.Label(newWindow, image = img_cs).place(x=5, y=300)
    panel.pack()

# To define and display a part of automaton of total event size = 'n'
def drawAutomaton2(root):
    global img_ps
    tempStorage = {}
    s2 = listify(parentState.get())
    n = totalEvents.get()
    ss = lowerState(count(n),s2, tempStorage)
    nfa = VisualNFA(
    states= set(allState(clean(ss))),
    input_symbols={'.'},
    transitions=transitionULD(ss),
    initial_state=initState(allState(clean(ss))),
    final_states={fstate},
    )
    nfa = VisualNFA(nfa)
    nfa.show_diagram().render("subAutomaton2", format='jpg')
    pdfConvertor() 
    newWindow1 = Toplevel(root)
    newWindow1.title("Automaton for" + " "+ parentState.get())
    newWindow1.geometry("1500x1000+30+30")
    Label(newWindow1, text = "The Transition Table:", font="Calibri 15").place(x=5, y=50)
    Label(newWindow1, text = nfa.table, font="Calibri 10").place(x=5, y=125)
    Label(newWindow1, text = "The Transition Diagram", font="Calibri 15").place(x=5, y=250)
    try:
        image = Image.open("C:/Users/joshu/subAutomaton2.jpg")
        image = image.resize((500, 450), Image.ANTIALIAS)
        img_ps = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(newWindow1, image = img_ps).place(x=5, y=300)
    panel.pack()

'''
SECTION 3:
1. Gap Implementation.
'''
'''
PART 4:
Function definitions to construct gap implementations
'''
def superpose(string1, string2, voc1, voc2):
    if len(string1) == len(string2) == 0:
        return [string1]
    else:
        temp = []
        if len(string1) > 0:
            h1 = string1[0]
            t1 = string1[1:]
            h1v2 = h1.intersection(voc2)
            if len(h1v2) == 0:
                for res in superpose(t1,string2,voc1,voc2):
                    temp.append([h1] + res)                  
            if len(string2) > 0:
                h2 = string2[0]
                t2 = string2[1:]
                h3 = h1.union(h2)
                if h1v2.issubset(h2):
                    h2v1 = h2.intersection(voc1)
                    if h2v1.issubset(h1):
                        for res in superpose(t1,t2,voc1,voc2):
                            temp.append([h3] + res)
        if len(string2) > 0:
            h2 = string2[0]
            t2 = string2[1:]
            h2v1 = h2.intersection(voc1)
            if len(h2v1) == 0:
                for res in superpose(string1,t2,voc1,voc2):
                    temp.append([h2] + res)
        return temp

# To define main function call.
def superposeString(s1,s2):
    return superpose(s1,s2,vocab(s1),vocab(s2))

# To extract all the vocabularies present inside the input string.
def vocab(string): 
    if len(string) == 0:
        return {}
    else:
        return string[0].union(vocab(string[1:]))

# To find Proper Subset.
def properPowerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)))

# To find Subset.
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# To implement Depadding.
def depadString(str): 
    if len(str) < 1:
        return str
    else:
        h,t = str[0],str[1:]
        if len(h) == 0:
            return depadString(t)
        else:
            return [h] + depadString(t)
# To implement A-Reduct.
def red(str,set): 
    if len(str) == 0:
        return str
    else:
        #lts= ' '.join([str(elem) for elem in str])
        h1 = str[0].intersection(set)
        return [h1] + red(str[1:],set)

'''
Function definitions to implement gap test function.
'''
def test(X,string1,string2):
    temp = []
    temp1 = []
    for i in X:
        #print ("For member:", i)
        A = vocab(i)
        B = list(powerset(A))
        for j in B:
            #print ("For subset:",j)
            c = red(i,j)
            D = depadString(c)
            E = superposeString(D,string2)
            #print ("superpose is",E)
            counter = 0
            for k in E:
                F = red(k,vocab(string1))
                G = depadString(F) 
                #print("output:",G)
                if G == string1:
                    counter = counter+1
            if counter == len(E):
                if D not in temp:
                    temp.append(D)              
        #print ('end')
    for i in temp:
        v = vocab(i)
        #print (v)
        subset = list(properPowerset(v))
        counterf = 0
        for j in subset:
            reduct = red(i,j)
            #print ("reduct",reduct)
            depaded = depadString(reduct)
            #print ("depaded",depaded)
            if depaded != string1:
                counterf = counterf+1
        if counterf == len(subset):
            if i not in temp1:
                temp1.append(i)
    return temp1
'''
VISUALIZATION PART 4:
Initializing functions responsible for visualization.
''' 
def gapImplementationWindow(root):
    newWindow4 = Toplevel(root)
    newWindow4.geometry("1500x1000+30+30")
    newWindow4.title("Gap Implementation for the given two strings")
    canvas = Canvas(newWindow4)
    fr = Frame(newWindow4, height=400, width=1350, highlightbackground="blue", highlightthickness=1)
    fr.pack_propagate(0) # don't shrink
    fr.place(x=75, y=150)
    Label(newWindow4, text="ENTER THE STRING REPRESENTATION OF THE EVENTS:", font="Calibri 12").place(x=10, y=0)  #label
    Label(newWindow4, text="Enter string 1:").grid(row = 0, padx=(10, 10), pady = (25,0))  #label
    Entry(newWindow4, textvariable = inputString1, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=0, column=3, pady = (25,0)) #entry textbox
    Entry(newWindow4, textvariable = inputString2, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=0, column=4, pady = (25,0)) #entry textbox
    Label(newWindow4, text="Enter string 2:").grid(row = 1, padx=(10, 10), pady = (25,0))  #labelEntry(root, textvariable = inputString2, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=1, column=3, pady = (25,0)) #entry textbox
    Entry(newWindow4, textvariable = inputString3, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=1, column=3, pady = (25,0)) #entry textbox
    Entry(newWindow4, textvariable = inputString4, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=1, column=4, pady = (25,0)) #entry textbox
    button1 = Button(newWindow4, text="Get the gap of given string combinations", command= lambda: retrieveGap(fr)).grid(row=3, column=0, padx=(10, 10), sticky=W)
    button2 = Button(newWindow4, text="Clear", command= lambda: clearCanvas(fr)).place(x=10, y = 200)

# To trigger and display superpose and gap immplementation.
def retrieveGap(root):
    string1 = []
    string2 = []
    string1Final = []
    string2Final = []
    string1.extend(inputString1.get())
    string1.extend(inputString2.get())
    string2.extend(inputString3.get())
    string2.extend(inputString4.get())
    for i in string1:
        f = {i}
        type(f)
        string1Final.append(f)
    for i in string2:
        f = {i}
        type(f)
        string2Final.append(f)    
    # Superpose implementation.
    Label(root, text="SUPERPOSE OF THE GIVEN STRING IS:", font="Calibri 12").place(x=10, y=10)  #label
    superposed = superposeString(string1Final,string2Final)
    for i, s in enumerate(superposed):
        display_str = str(s)
        f = Frame(root, height=20, width=200)
        f.pack_propagate(0) # don't shrink
        f.place(x=10, y=60+20*i)
        label = Label(f, text=display_str, borderwidth=1, relief="solid", bg='linen')  #label
        label.pack(fill=BOTH, expand=1)

    # Gap implementation.
    Label(root, text="GAP OF THE GIVEN STRINGS ARE:", font="Calibri 12").place(x=10, y=150)  #label
    result = test(superposed,string1Final,string2Final)
    for i, s in enumerate(result):
        display_str1 = str(s)
        f1 = Frame(root, height=20, width=200)
        f1.pack_propagate(0) # don't shrink
        f1.place(x=10, y=200+20*i)
        label = Label(f1, text=display_str1, borderwidth=1, relief="solid", bg='linen')  #label
        label.pack(fill=BOTH, expand=1)
        

'''
Initializing main tkinter and canvas
'''
root = Tk()
root.geometry("1500x1000+30+30")
root.title("Defining & Superposiing two interval Automaton based on {U, L, D}")
canvas = Canvas(root)
# Part 1A and 1B input variables.
eventNumber1 = IntVar()
eventNumber2 = IntVar()
transState = StringVar()
transPath = StringVar()
noOfEvents = IntVar()
startState = StringVar()
# Part 2 input variables.
allenFirstRel = StringVar()
allenSecondRel = StringVar()
projectionReq = StringVar()
# Part 3: Input Variables.
totalEvents = IntVar()
childState = StringVar()
parentState = StringVar()
breakValue = StringVar()
# Part 4: Input Variables.
inputString1 = StringVar()
inputString2 = StringVar()
inputString3 = StringVar()
inputString4 = StringVar()

Label(root, text="ENTER THE EVENT NUMBER:", font="Calibri 12").place(x=5, y=0)  #label
Label(root, text="FIRST:").grid(row = 0, padx=(10, 10), pady = (25,0))  #label
Label(root, text="SECOND:").grid(row = 1, padx=(10, 10), pady = (25,0))  #label
Entry(root, textvariable = eventNumber1, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=0, column=3, pady = (25,0)) #entry textbox
Entry(root, textvariable = eventNumber2, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=1, column=3, pady = (25,0)) #entry textbox

fr = Frame(root, height=365, width=1350, highlightbackground="blue", highlightthickness=1)
fr.pack_propagate(0) # don't shrink
fr.place(x=90, y=100)

WSignUp1 = Button(root, text="First event ULD", command= lambda: showAutomaton1(fr)).grid(row=0, column=5, padx=(10, 10), sticky=W)
WSignUp2 = Button(root, text="Second event ULD", command= lambda: showAutomaton2(fr)).grid(row=1, column=5, padx=(10, 10), sticky=W)
button = Button(root, text="Clear", command= lambda: clearCanvas(fr)).grid(row=3, column=0, padx=(10, 10),  sticky=W)

Label(root, text="SUPERPOSING THE TWO INTERVALS:", font="Calibri 12").place(x=10, y=500)  #label
fr1 = Frame(root, height=500, width=1140, highlightbackground="blue", highlightthickness=1)
fr1.pack_propagate(0) # don't shrink
fr1.place(x=300, y=470)
button1 = Button(root, text="Display Superposed automaton of event1 and event2", command= lambda: displaySuperposition(fr1)).place(x=5, y = 520)
Label(root, text="ENTER STATE RECORDS SEQUENCE:").place(x = 5, y = 550)   #label
Entry(root, textvariable = transState, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 580)#entry textbox
WSignUp1 = Button(root, text="Check for Acceptance", command = lambda: acceptanceStateCheck(root)).place(x=5, y = 610)
button1 = Button(root, text="Clear", command= lambda: clearCanvas(fr1)).place(x=5, y = 645)
button1 = Button(root, text="Superposing more than 1 events", command= lambda: showSuperposeIntervalWindow(root)).place(x=5, y = 675)
root.mainloop()
{"mode":"full","isActive":False}

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\joshu\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-2-3033e831f931>", line 799, in <lambda>
    Wdisplay1 = Button(root, text="Display the Sword automaton", command= lambda: displayAutomaton1(fr, fa1)).place(x=0, y=150)
  File "<ipython-input-2-3033e831f931>", line 829, in displayAutomaton1
    panel.pack()
AttributeError: 'NoneType' object has no attribute 'pack'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\joshu\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-2-3033e831f931>", line 844, in <lambda>
    Wdisplay1 = Button(root, text="Display the Sword automaton", command= lambda: displayAutomaton2(fr, fa2)).place(x=600, y=150)
  File "<ipython-input-2-3033e831f931>", line 875, in displayAutomaton2
    panel.pack()
AttributeError: 'NoneType' obj

[Accepted]                                 
Step:     Current state: Input symbol:         New state:
1      →{2- 'u', 1- 'u'}        [1, 2]   {2- 'l', 1- 'l'}
2       {2- 'l', 1- 'l'}      [-2, -1]  *{2- 'd', 1- 'd'}

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\joshu\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-2-3033e831f931>", line 947, in <lambda>
    button2 = Button(newWindow1, text="Dispay Acceptance", command= lambda: displayPathAcceptance(newWindow1)).place(x=700, y = 60)
  File "<ipython-input-2-3033e831f931>", line 1044, in displayPathAcceptance
    panel.pack()
AttributeError: 'NoneType' object has no attribute 'pack'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\joshu\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-2-3033e831f931>", line 1062, in <lambda>
    button2 = Button(newWindow2, text="Display Automaton", command= lambda: displayAllenSecondRel(newWindow2)).place(x=700, y = 60)
  File "<ipython-input-2-3033e831f931>", line 1134, in displayAllenSecondRel
    panel.pack()
AttributeErro

{'mode': 'full', 'isActive': False}